In [2]:
import sys
import pandas as pd
from typing import List, Dict
from SPARQLWrapper import SPARQLWrapper, JSON

class WikiDataQueryResults:
    """
    A class that can be used to query data from Wikidata using SPARQL and return the results as a Pandas DataFrame or a list
    of values for a specific key.
    """
    def __init__(self, query: str):
        """
        Initializes the WikiDataQueryResults object with a SPARQL query string.

        :param query: A SPARQL query string.
        """
        self.user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
        self.endpoint_url = "https://query.wikidata.org/sparql"
        self.sparql = SPARQLWrapper(self.endpoint_url, agent=self.user_agent)
        self.sparql.setQuery(query)
        self.sparql.setReturnFormat(JSON)

    def __transform2dicts(self, results: List[Dict]) -> List[Dict]:
        """
        Helper function to transform SPARQL query results into a list of dictionaries.

        :param results: A list of query results returned by SPARQLWrapper.
        :return: A list of dictionaries, where each dictionary represents a result row and has keys corresponding to the
        variables in the SPARQL SELECT clause.
        """
        new_results = []
        for result in results:
            new_result = {}
            for key in result:
                new_result[key] = result[key]['value']
            new_results.append(new_result)
        return new_results

    def _load(self) -> List[Dict]:
        """
        Helper function that loads the data from Wikidata using the SPARQLWrapper library, and transforms the results into
        a list of dictionaries.

        :return: A list of dictionaries, where each dictionary represents a result row and has keys corresponding to the
        variables in the SPARQL SELECT clause.
        """
        results = self.sparql.queryAndConvert()['results']['bindings']
        results = self.__transform2dicts(results)
        return results

    def load_as_dataframe(self) -> pd.DataFrame:
        """
        Executes the SPARQL query and returns the results as a Pandas DataFrame.

        :return: A Pandas DataFrame representing the query results.
        """
        results = self._load()
        return pd.DataFrame.from_dict(results)

In [4]:
query = """#Places of residence of accused witches in Scotland 1563-1736
SELECT ?item ?witch ?itemLabel ?residenceLabel ?coordinate ?genderLabel ?occupationLabel ?chargeLabel ?classLabel ?ethnicLabel ?mdeathLabel ?cdeathLabel ?deathlocLabel ?detainLabel
WHERE {
  ?item wdt:P4478 ?witch;
    wdt:P551 ?residence.
  ?residence wdt:P625 ?coordinate.
  OPTIONAL 
  {
    ?item wdt:P21 ?gender .
  }
  OPTIONAL
  {
    ?item wdt:P106 ?occupation .
  }
  OPTIONAL 
  {
    #?item wdt:P1595 ?charge . 
  }
  OPTIONAL
  {
    ?item wdt:P3716 ?class . 
  }
  OPTIONAL 
  {
    ?item wdt:P172 ?ethnic .
  }
  OPTIONAL 
  {
    ?item wdt:P1196 ?mdeath . 
  } 
  OPTIONAL 
  { 
    #?item wdt:P509 ?cdeath .
  }
  OPTIONAL 
  {
    ?item wdt:P20 ?deathloc . 
   } 
  OPTIONAL 
  {
    ?item wdt:P2632 ?detain . 
  }
 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}"""

data_extracter = WikiDataQueryResults(query)
df = data_extracter.load_as_dataframe()

                                       item      witch  \
0  http://www.wikidata.org/entity/Q43389933  A/EGD/947   
1  http://www.wikidata.org/entity/Q43390990  A/EGD/385   
2  http://www.wikidata.org/entity/Q43390997  A/EGD/391   
3  http://www.wikidata.org/entity/Q43391058  A/EGD/443   
4  http://www.wikidata.org/entity/Q43391069  A/EGD/454   

                   coordinate        itemLabel residenceLabel genderLabel  \
0      Point(-3.6293 56.0554)     Helene Ezatt        Culross      female   
1     Point(-3.14673 55.9416)    Issobel Broun    Duddingston      female   
2     Point(-3.14673 55.9416)    Isobel Ramsay    Duddingston      female   
3  Point(-3.783964 57.583936)  Elspet Falconer         Penick      female   
4  Point(-3.753976 57.556804)        Janet Man    East Milton      female   

  detainLabel classLabel         mdeathLabel deathlocLabel occupationLabel  \
0     Culross        NaN                 NaN           NaN             NaN   
1         NaN   middling        

In [7]:
df

,item,witch,coordinate,itemLabel,residenceLabel,genderLabel,detainLabel,classLabel,mdeathLabel,deathlocLabel,occupationLabel,ethnicLabel
0,http://www.wikidata.org/entity/Q43389933,A/EGD/947,Point(-3.6293 56.0554),Helene Ezatt,Culross,female,Culross,NaN,NaN,NaN,NaN,NaN
1,http://www.wikidata.org/entity/Q43390990,A/EGD/385,Point(-3.14673 55.9416),Issobel Broun,Duddingston,female,NaN,middling,NaN,NaN,NaN,NaN
2,http://www.wikidata.org/entity/Q43390997,A/EGD/391,Point(-3.14673 55.9416),Isobel Ramsay,Duddingston,female,NaN,NaN,capital punishment,NaN,NaN,NaN
3,http://www.wikidata.org/entity/Q43391058,A/EGD/443,Point(-3.783964 57.583936),Elspet Falconer,Penick,female,NaN,NaN,NaN,NaN,NaN,NaN
4,http://www.wikidata.org/entity/Q43391069,A/EGD/454,Point(-3.753976 57.556804),Janet Man,East Milton,female,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3197,http://www.wikidata.org/entity/Q43396206,A/LA/3173,Point(-2.522777777 55.998333333),Agnes Bathcut,Dunbar,female,NaN,NaN,NaN,NaN,NaN,NaN
3198,http://www.wikidata.org/entity/Q43396227,A/LA/3222,Point(-2.52264 56.5895),Henrye Stevin,Auchmithie,male,NaN,NaN,NaN,NaN,NaN,NaN
3199,http://www.wikidata.org/entity/Q43396240,A/LA/3237,Point(-3.196791 56.584612),Lyvelyke,Kinloch,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3200,http://www.wikidata.org/entity/Q55641024,A/EGD/1327,Point(-4.62856 55.7925),Mauld Gauld,Lochwinnoch,female,NaN,middling,NaN,NaN,NaN,NaN


In [6]:
df[df['itemLabel']=='Niniane Chirneyside']

,item,witch,coordinate,itemLabel,residenceLabel,genderLabel,detainLabel,classLabel,mdeathLabel,deathlocLabel,occupationLabel,ethnicLabel
620,http://www.wikidata.org/entity/Q43392555,A/EGD/102,Point(-3.189166666 55.953333333),Niniane Chirneyside,Edinburgh,male,NaN,middling,NaN,NaN,servant,NaN
